In [ ]:
import pandas as pd
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

# 한국동서발전 시간대별 태양광 발전량 데이터 추가

발전소명

* 당진태양광 : dangjin
* 당진수상태양광 : dangjin_floating
* 당진자재창고태양광 : dangjin_warehouse
* 울산태양광 : ulsan

태양광 발전량 데이터 경로 및 파일 이름 : /data/
* 한국동서발전 시간대별 태양광 발전량 현황(2015_2017).csv
* 한국동서발전 시간대별 태양광 및 풍력 발전량 현황(2018_2019).csv

태양광 발전량 데이터도 결측값이 있습니다.
* 울산태양광 15년 1월 2일 ~ 1월 31일
 


In [ ]:
def convert_time(x):
    Ymd, HMS = x.split(' ')
    H, M, S = HMS.split(':')
    H = str(int(H)-1)
    HMS = ':'.join([H, M, S])
    return ' '.join([Ymd, HMS])  

## 기존 태양광 발전량 데이터

In [ ]:
energy = pd.read_csv('./data/energy.csv')
energy['time'] = energy['time'].apply(lambda x:convert_time(x))
energy['time']=pd.to_datetime(energy['time']) + pd.DateOffset(hours=1)
energy[['dangjin','ulsan']]=energy[['dangjin','ulsan']].astype(float)
print(energy.info())
energy

## 추가할 2015년 1월 ~ 2018년 2월 태양광 발전량 데이터

In [ ]:
past_energy1 = pd.read_csv('./data/한국동서발전 시간대별 태양광 및 풍력 발전량 현황(2018_2019).csv', encoding='cp949')
past_energy2 = pd.read_csv('./data/한국동서발전 시간대별 태양광 발전량 현황(2015_2017).csv', encoding='cp949')
past_energy1['시간']=pd.to_datetime(past_energy1['시간'])
past_energy2['시간']=pd.to_datetime(past_energy2['시간'])
past_energy=pd.concat([past_energy1,past_energy2])
past_energy.sort_values(by=['시간'],ascending=True, inplace=True)

dangjin_energy=past_energy[past_energy['태양광명']=='당진태양광']
dangjin_float_energy=past_energy[(past_energy['태양광명']=='당진수상태양광')|(past_energy['태양광명']=='당진취수로태양광')] 
dangjin_warehouse_energy=past_energy[past_energy['태양광명']=='당진자재창고태양광']
ulsan_energy=past_energy[past_energy['태양광명']=='울산태양광']

## 필요한 부분만 잘라내기

In [ ]:
ulsan_energy=ulsan_energy[ulsan_energy['시간'].dt.year <= 2018]
ulsan_energy=ulsan_energy[~((ulsan_energy['시간'].dt.year == 2018) & (ulsan_energy['시간'].dt.month >= 3))]
dangjin_energy=dangjin_energy[dangjin_energy['시간'].dt.year <= 2018]
dangjin_energy=dangjin_energy[~((dangjin_energy['시간'].dt.year == 2018) & (dangjin_energy['시간'].dt.month >= 3))]
dangjin_float_energy=dangjin_float_energy[dangjin_float_energy['시간'].dt.year <= 2018]
dangjin_float_energy=dangjin_float_energy[~((dangjin_float_energy['시간'].dt.year == 2018) & (dangjin_float_energy['시간'].dt.month >= 3))]
dangjin_warehouse_energy=dangjin_warehouse_energy[dangjin_warehouse_energy['시간'].dt.year <= 2018]
dangjin_warehouse_energy=dangjin_warehouse_energy[~((dangjin_warehouse_energy['시간'].dt.year == 2018) & (dangjin_warehouse_energy['시간'].dt.month >= 3))]

## 시간대별로 만들기위해 일별로 24번 반복

In [ ]:
ulsan_energy=ulsan_energy.reset_index(drop=True)
dangjin_energy=dangjin_energy.reset_index(drop=True)
dangjin_float_energy=dangjin_float_energy.reset_index(drop=True)
dangjin_warehouse_energy=dangjin_warehouse_energy.reset_index(drop=True)

dangjin=pd.DataFrame(columns=['시간','dangjin'])
dangjin_floating=pd.DataFrame(columns=['시간','dangjin_floating'])
dangjin_warehouse=pd.DataFrame(columns=['시간','dangjin_warehouse'])
ulsan=pd.DataFrame(columns=['시간','ulsan'])

dangjin['dangjin']=dangjin['dangjin'].astype(float)
dangjin_floating['dangjin_floating']=dangjin_floating['dangjin_floating'].astype(float)
dangjin_warehouse['dangjin_warehouse']=dangjin_warehouse['dangjin_warehouse'].astype(float)
ulsan['ulsan']=ulsan['ulsan'].astype(float)

for i in range(24):
    dangjin=dangjin.append(dangjin_energy,ignore_index = True)
    dangjin_floating=dangjin_floating.append(dangjin_float_energy,ignore_index = True)
    dangjin_warehouse=dangjin_warehouse.append(dangjin_warehouse_energy,ignore_index = True)
    ulsan=ulsan.append(ulsan_energy,ignore_index = True)

dangjin.sort_values(by=['시간'],ascending=True, inplace=True)
dangjin_floating.sort_values(by=['시간'],ascending=True, inplace=True)
dangjin_warehouse.sort_values(by=['시간'],ascending=True, inplace=True)
ulsan.sort_values(by=['시간'],ascending=True, inplace=True)

dangjin.reset_index(drop=True, inplace=True)
dangjin_floating.reset_index(drop=True, inplace=True)
dangjin_warehouse.reset_index(drop=True, inplace=True)
ulsan.reset_index(drop=True, inplace=True)

## 시간별로 변환

In [ ]:
for i in range(24):
    dangjin.iloc[i::24,0]=pd.to_datetime('20' + dangjin.iloc[i::24,0].dt.strftime('%y-%m-%d')+ ' ' + str(i) + ':00') + pd.DateOffset(hours=1)
    dangjin_floating.iloc[i::24,0]=pd.to_datetime('20' + dangjin_floating.iloc[i::24,0].dt.strftime('%y-%m-%d')+ ' ' + str(i) + ':00') + pd.DateOffset(hours=1)
    dangjin_warehouse.iloc[i::24,0]=pd.to_datetime('20' + dangjin_warehouse.iloc[i::24,0].dt.strftime('%y-%m-%d')+ ' ' + str(i) + ':00') + pd.DateOffset(hours=1)
    ulsan.iloc[i::24,0]=pd.to_datetime('20' + ulsan.iloc[i::24,0].dt.strftime('%y-%m-%d')+ ' ' + str(i) + ':00') + pd.DateOffset(hours=1)

    dangjin.iloc[i::24,1]=dangjin.iloc[i::24,i+3].astype(float)
    dangjin_floating.iloc[i::24,1]=dangjin_floating.iloc[i::24,i+3].astype(float)
    dangjin_warehouse.iloc[i::24,1]=dangjin_warehouse.iloc[i::24,i+3].astype(float)
    ulsan.iloc[i::24,1]=ulsan.iloc[i::24,i+3]

dangjin=dangjin.iloc[:,0:2]
dangjin_floating=dangjin_floating.iloc[:,0:2]
dangjin_warehouse=dangjin_warehouse.iloc[:,0:2]
ulsan=ulsan.iloc[:,0:2]
dangjin.columns = ['time', 'dangjin']
dangjin_floating.columns = ['time', 'dangjin_floating']
dangjin_warehouse.columns = ['time', 'dangjin_warehouse']
ulsan.columns = ['time', 'ulsan']

In [ ]:
ulsan.info()

## 하나의 데이터프레임으로 합치기

In [ ]:
new_energy= pd.DataFrame(columns=['time'])
new_energy['time'] = pd.date_range(start='2015-01-01 01:00:00', end='2018-03-01 00:00:00', freq='H')
new_energy = pd.merge(new_energy, dangjin_floating, on=['time'], how='outer')
new_energy = pd.merge(new_energy, dangjin_warehouse, on=['time'],how='outer')
new_energy = pd.merge(new_energy, dangjin, on=['time'], how='outer')
new_energy = pd.merge(new_energy, ulsan, on=['time'],how='outer')
new_energy = pd.concat([new_energy, energy])

In [ ]:
new_energy.head(10)

In [ ]:
new_energy.to_csv('./new_data/new_energy.csv')